# Regresión lineal univariable #

### Predicción de precios de casas con la implementación de un modelo de regresión lineal ###

El presente proyecto consiste en realizar un modelo de de regresión lineal univariable para la predicción de precios de apartamentos. La implementación del modelo será totalmente manual, sin utilizar Scikit-learn para su entrenamiento. 


### Modelo de regresión lineal ###

Un modelo de regresión lineal busca encontrar la ecuación lineal que minimice el error cuadrado medio.

La ecuación lineal:

$f(x) = y = mx + b$

donde: 

$y$ : Variable dependiente
$x$ : Variable independiente
$m$ : Pendiente
$b$ : Intercepto

Error cuadrado medio:

$f(m,b) = \frac{1}{N} \sum_{i=1}^{n} (y_{i}−(mx_{i}+b))^2$

donde:

* $y_{i}$ : Valor real de cada dato en el dataset
* $mx_{i}+b$ : Valor predecido por el modelo

Para poder optimizar este modelo es necesario encontrar los valores de $m$ y $b$ que minimizan la función de costo.
Esto se realiza a través de un aprendizaje por medio de *gradient descent*.

<img src="regresionlineal.png"
     alt="regresionlineal"
     style="float: left; margin-right: 10px;" />




In [4]:
# Importación de librerías a utilizar

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sklearn
import seaborn as sb

In [6]:
#Cargar dataset

np.load('proyecto_training_data.npy')

array([[2.08500e+05, 7.00000e+00, 8.56000e+02, 8.00000e+00, 2.00300e+03,
        6.50000e+01],
       [1.81500e+05, 6.00000e+00, 1.26200e+03, 6.00000e+00, 1.97600e+03,
        8.00000e+01],
       [2.23500e+05, 7.00000e+00, 9.20000e+02, 6.00000e+00, 2.00100e+03,
        6.80000e+01],
       ...,
       [2.66500e+05, 7.00000e+00, 1.18800e+03, 9.00000e+00, 1.94100e+03,
        6.60000e+01],
       [1.42125e+05, 5.00000e+00, 1.07800e+03, 5.00000e+00, 1.95000e+03,
        6.80000e+01],
       [1.47500e+05, 5.00000e+00, 1.25600e+03, 6.00000e+00, 1.96500e+03,
        7.50000e+01]])